In [ ]:
# """
# Hyperparameters can be efficiently tuned with `optuna <https://optuna.readthedocs.io/>`_.
# """
# import copy
# import logging
# import os
# from typing import Any, Dict, Tuple, Union

# import numpy as np
# import optuna
# from optuna.integration import PyTorchLightningPruningCallback, TensorBoardCallback
# import optuna.logging
# import pytorch_lightning as pl
# from pytorch_lightning import Callback
# from pytorch_lightning.callbacks import LearningRateMonitor
# from pytorch_lightning.loggers import TensorBoardLogger
# import statsmodels.api as sm
# import torch
# from torch.utils.data import DataLoader

# from pytorch_forecasting import TemporalFusionTransformer, RecurrentNetwork
# from pytorch_forecasting.data import TimeSeriesDataSet
# from pytorch_forecasting.metrics import QuantileLoss

# optuna_logger = logging.getLogger("optuna")

# class MetricsCallback(Callback):
#     """PyTorch Lightning metric callback."""

#     def __init__(self):
#         super().__init__()
#         self.metrics = []

#     def on_validation_end(self, trainer, pl_module):
#         self.metrics.append(trainer.callback_metrics)

In [ ]:
# def optimize_hyperparameters_for_rnn(
#     train_dataloaders: DataLoader,
#     val_dataloaders: DataLoader,
#     model_path: str,
#     max_epochs: int = 20,
#     n_trials: int = 100,
#     timeout: float = 3600 * 8.0,  # 8 hours
#     gradient_clip_val_range: Tuple[float, float] = (0.01, 100.0),
#     hidden_size_range: Tuple[int, int] = (16, 265),
#     # hidden_continuous_size_range: Tuple[int, int] = (8, 64),
#     # attention_head_size_range: Tuple[int, int] = (1, 4),
#     dropout_range: Tuple[float, float] = (0.1, 0.3),
#     learning_rate_range: Tuple[float, float] = (1e-5, 1.0),
#     use_learning_rate_finder: bool = True,
#     trainer_kwargs: Dict[str, Any] = {},
#     log_dir: str = "lightning_logs",
#     study: optuna.Study = None,
#     verbose: Union[int, bool] = None,
#     pruner: optuna.pruners.BasePruner = optuna.pruners.SuccessiveHalvingPruner(),
#     **kwargs,
# ) -> optuna.Study:

#     assert isinstance(train_dataloaders.dataset, TimeSeriesDataSet) and isinstance(
#         val_dataloaders.dataset, TimeSeriesDataSet
#     ), "dataloaders must be built from timeseriesdataset"

#     logging_level = {
#         None: optuna.logging.get_verbosity(),
#         0: optuna.logging.WARNING,
#         1: optuna.logging.INFO,
#         2: optuna.logging.DEBUG,
#     }
#     optuna_verbose = logging_level[verbose]
#     optuna.logging.set_verbosity(optuna_verbose)

#     loss = kwargs.get(
#         "loss", QuantileLoss()
#     )  # need a deepcopy of loss as it will otherwise propagate from one trial to the next

#     # create objective function
#     def objective(trial: optuna.Trial) -> float:
#         # Filenames for each trial must be made unique in order to access each checkpoint.
#         checkpoint_callback = pl.callbacks.ModelCheckpoint(
#             dirpath=os.path.join(model_path, "trial_{}".format(trial.number)), filename="{epoch}", monitor="val_loss"
#         )

#         # The default logger in PyTorch Lightning writes to event files to be consumed by
#         # TensorBoard. We don't use any logger here as it requires us to implement several abstract
#         # methods. Instead we setup a simple callback, that saves metrics from each validation step.
#         metrics_callback = MetricsCallback()
#         learning_rate_callback = LearningRateMonitor()
#         logger = TensorBoardLogger(log_dir, name="optuna", version=trial.number)
#         gradient_clip_val = trial.suggest_loguniform("gradient_clip_val", *gradient_clip_val_range)
#         default_trainer_kwargs = dict(
#             gpus=[0] if torch.cuda.is_available() else None,
#             max_epochs=max_epochs,
#             gradient_clip_val=gradient_clip_val,
#             callbacks=[
#                 metrics_callback,
#                 learning_rate_callback,
#                 checkpoint_callback,
#                 PyTorchLightningPruningCallback(trial, monitor="val_loss"),
#             ],
#             logger=logger,
#             enable_progress_bar=optuna_verbose < optuna.logging.INFO,
#             weights_summary=[None, "top"][optuna_verbose < optuna.logging.INFO],
#         )
#         default_trainer_kwargs.update(trainer_kwargs)
#         trainer = pl.Trainer(
#             **default_trainer_kwargs,
#         )

#         # create model
#         hidden_size = trial.suggest_int("hidden_size", *hidden_size_range, log=True)
#         kwargs["loss"] = copy.deepcopy(loss)
#         model = RecurrentNetwork.from_dataset(
#             train_dataloaders.dataset,
#             dropout=trial.suggest_uniform("dropout", *dropout_range),
#             hidden_size=hidden_size,
#             # hidden_continuous_size=trial.suggest_int(
#             #     "hidden_continuous_size",
#             #     hidden_continuous_size_range[0],
#             #     min(hidden_continuous_size_range[1], hidden_size),
#             #     log=True,
#             # ),
#             # attention_head_size=trial.suggest_int("attention_head_size", *attention_head_size_range),
#             log_interval=-1,
#             **kwargs,
#         )
#         # find good learning rate
#         if use_learning_rate_finder:
#             lr_trainer = pl.Trainer(
#                 gradient_clip_val=gradient_clip_val,
#                 gpus=[0] if torch.cuda.is_available() else None,
#                 logger=False,
#                 enable_progress_bar=False,
#                 enable_model_summary=False,
#             )
#             res = lr_trainer.tuner.lr_find(
#                 model,
#                 train_dataloaders=train_dataloaders,
#                 val_dataloaders=val_dataloaders,
#                 early_stop_threshold=10000,
#                 min_lr=learning_rate_range[0],
#                 num_training=100,
#                 max_lr=learning_rate_range[1],
#             )

#             loss_finite = np.isfinite(res.results["loss"])
#             if loss_finite.sum() > 3:  # at least 3 valid values required for learning rate finder
#                 lr_smoothed, loss_smoothed = sm.nonparametric.lowess(
#                     np.asarray(res.results["loss"])[loss_finite],
#                     np.asarray(res.results["lr"])[loss_finite],
#                     frac=1.0 / 10.0,
#                 )[min(loss_finite.sum() - 3, 10) : -1].T
#                 optimal_idx = np.gradient(loss_smoothed).argmin()
#                 optimal_lr = lr_smoothed[optimal_idx]
#             else:
#                 optimal_idx = np.asarray(res.results["loss"]).argmin()
#                 optimal_lr = res.results["lr"][optimal_idx]
#             optuna_logger.info(f"Using learning rate of {optimal_lr:.3g}")
#             # add learning rate artificially
#             model.hparams.learning_rate = trial.suggest_uniform("learning_rate", optimal_lr, optimal_lr)
#         else:
#             model.hparams.learning_rate = trial.suggest_loguniform("learning_rate", *learning_rate_range)

#         # fit
#         trainer.fit(model, train_dataloaders=train_dataloaders, val_dataloaders=val_dataloaders)

#         # report result
#         return metrics_callback.metrics[-1]["val_loss"].item()

#     # setup optuna and run
#     if study is None:
#         study = optuna.create_study(direction="minimize", pruner=pruner)
#     study.optimize(objective, n_trials=n_trials, timeout=timeout)
#     return study

In [ ]:
import torch, json
from math import floor
import pandas as pd
import numpy as np

from FileManager.dataManager import dataManager
from AnalyzeTools.models import autoregressive_integrated_moving_average, linear_regression, support_vector_regression, random_forest, gradient_boosting
from AnalyzeTools.prepare import data_split, model_eval, pathForSavingModels
from AnalyzeTools.preprocess import preprocessData
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

params_path = './Models/single'
train_size = 0.8
product_object = json.load(open("./File information.json", "r", encoding='utf8'))

all_experiments= []
for product in product_object.keys():
    for raw_file_name in  product_object[product].keys():
        for product_type in product_object[product][raw_file_name]['product_types']:
            for target in product_object[product][raw_file_name]['targets']:
                all_experiments.append([product, raw_file_name, product_type, target])

all_experiments

In [ ]:
n = -1
experiment = all_experiments[n]
product, raw_file_name, product_type, target = experiment
print(f"Product: {product}\nRaw file name: {raw_file_name}\nProduct_type: {product_type}\ntarget: {target}")

In [ ]:
df, product_and_product_type, product_attribute = dataManager(raw_file_name, product, product_type, target)

if len(df) == 0:
    raise ValueError("No data!")

df, input_features = preprocessData(df, 'date', target)
predictions_x_axis = df['date'][floor(len(df) * train_size):].values

In [ ]:
data = df.copy()

data['time_idx'] = range(len(data))
data['group'] = product

training_cutoff = floor(len(data) * 0.8)

max_prediction_length = 1
max_encoder_length = 30 # 7, 14, 30, 60, 120
batch_size = 64

group = ['group']
time_varying_known_categoricals = ['month', 'week']
time_varying_unknown_categoricals = []
time_varying_known_reals = ['time_idx']
time_varying_unknown_reals = input_features + [target]

In [ ]:
print("\nLSTM")
lstm, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'LSTM'),
    'LSTM'
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
lstm_predictions = lstm.predict(val_dataloader)
model_eval(actuals, lstm_predictions, predictions_x_axis, stdout=True, vis=True)

In [ ]:
# print("\nDeepAR")
# deep_ar, val_dataloader = DEEPAR(
#     data,
#     training_cutoff,
#     target,
#     group,
#     max_encoder_length,
#     max_prediction_length,
#     time_varying_known_categoricals,
#     time_varying_unknown_categoricals,
#     time_varying_known_reals,
#     batch_size,
#     pathForSavingModels(product_and_product_type, product_attribute, raw_file_name, 'DEEPAR'),
# )

# actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
# deepar_predictions = deep_ar.predict(val_dataloader)

# model_eval(actuals, deepar_predictions, predictions_x_axis, stdout=True, vis=True)